<a href="https://colab.research.google.com/github/Raghav0737/capstone-project-2025/blob/main/pricing_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# Dynamic Pricing Model Functions
# ================================

# Model 1: Baseline Linear Pricing
def baseline_linear_model(prev_price, occupancy, capacity, alpha=2.0):
    """Price increases linearly with occupancy"""
    return prev_price + alpha * (occupancy / capacity)

# ------------------------------
# Model 2: Demand-Based Pricing
# ------------------------------

VEHICLE_TYPE_WEIGHT = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
TRAFFIC_LEVEL_MAP = {'low': 0.5, 'medium': 1.0, 'high': 1.5}

def compute_demand(occupancy, capacity, queue_length, traffic, special_day, vehicle_type,
                   alpha=1.5, beta=1.0, gamma=1.2, delta=2.0, epsilon=1.5):
    """Computes a weighted demand score based on parking conditions"""
    occ_rate = occupancy / capacity
    traffic_score = TRAFFIC_LEVEL_MAP.get(traffic, 1.0)
    vehicle_weight = VEHICLE_TYPE_WEIGHT.get(vehicle_type, 1.0)
    return (alpha * occ_rate +
            beta * queue_length -
            gamma * traffic_score +
            delta * special_day +
            epsilon * vehicle_weight)

def demand_based_price(base_price, demand, lambda_=0.1):
    """Adjusts price using normalized demand"""
    norm_demand = (demand - 1) / 10  # Example normalization
    multiplier = max(0.5, min(2.0, 1 + lambda_ * norm_demand))
    return base_price * multiplier

# ------------------------------
# Model 3: Competitive Pricing
# ------------------------------

import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between two geo-coordinates in kilometers"""
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def adjust_price_with_competition(current_price, nearby_prices, is_full, competitor_threshold=0.1):
    """Adjust price based on competitor prices within 1km"""
    if not nearby_prices:
        return current_price

    avg_nearby_price = sum(nearby_prices) / len(nearby_prices)
    if is_full and avg_nearby_price < current_price:
        return max(10, current_price - competitor_threshold * (current_price - avg_nearby_price))
    elif avg_nearby_price > current_price:
        return min(20, current_price + competitor_threshold * (avg_nearby_price - current_price))
    return current_price
